In [ ]:
import numpy as np
import pandas as pd

import math
import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import tensorflow_probability as tfp
from keras.models import Sequential
from keras.layers import Dense, LSTM

from tensorflow.keras.layers import StringLookup

#ignore warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# brewer2mpl.get_map args: set name set type number of colors
import matplotlib as mpl

#mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)
mpl.rcParams['font.family'] = 'Times New Roman'

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

In [ ]:
data=pd.read_csv('data.csv')
item=3

data['time']=pd.to_datetime(data['time'],format = '%Y-%m-%d %H:%M:%S')

turbulence = np.zeros((len(data), 1))

speed_for_turbulence = np.array(data['speed(m/s)_%s'%item])

for i in range(5,len(data)-1):
  tmp = speed_for_turbulence[(i-5):(i+1)]
  turbulence[i][0] = np.std(tmp)/(np.mean(tmp)+0.001)

turbulence[turbulence>1] = 1
data['turbulence_%s'%item]=turbulence

angle=np.array(data['angle(°)_%s'%item])
cos_angle=[np.cos(np.pi*(i/180)) for i in angle]
data['angle(°)_%s'%item]=cos_angle

norm_data_original=data[data['state_%s'%item]==10]
norm_data1=norm_data_original[norm_data_original['active_value_%s'%item]==100]
norm_data2=norm_data_original[norm_data_original['active_value_%s'%item]==0]

norm_data=pd.concat([norm_data1,norm_data2])

ab_data=data[data['state_%s'%item]!=10]
limit_data=pd.concat([norm_data_original,norm_data],axis =0).drop_duplicates(keep=False)
limit_data=pd.concat([limit_data,ab_data])

speed_all=np.array(data['speed(m/s)_%s'%item])
power_all=np.array(data['power(kW)_%s'%item])
temperature_all=np.array(data['temperature(℃)_%s'%item])
angle_all=np.array(data['angle(°)_%s'%item])
turbulence_all=np.array(data['turbulence_%s'%item])


speed=np.array(norm_data['speed(m/s)_%s'%item])
power=np.array(norm_data['power(kW)_%s'%item])
temperature=np.array(norm_data['temperature(℃)_%s'%item])
angle=np.array(norm_data['angle(°)_%s'%item])
turbulence=np.array(norm_data['turbulence_%s'%item])

speed_index=norm_data['speed(m/s)_%s'%item].index 

#generate dataset
speed_data=[[] for i in range(120)]
power_data=[[] for i in range(120)]
tem_data=[[] for i in range(120)]
angle_data=[[] for i in range(120)]
turbulence_data=[[] for i in range(120)]

speed_index=norm_data['speed(m/s)_%s'%item].index
for i in speed_index:
    for j in range(120):
        speed_data[j].append(speed_all[i-j])

power_index=norm_data['power(kW)_%s'%item].index

for i in power_index:
    for j in range(120):
        power_data[j].append(power_all[i-j])
    
tem_index=norm_data['temperature(℃)_%s'%item].index

for i in tem_index:
    for j in range(120):
        tem_data[j].append(temperature_all[i-j])

angle_index=norm_data['angle(°)_%s'%item].index

for i in angle_index:
    for j in range(120):
        angle_data[j].append(angle_all[i-j])

turbulence_index=norm_data['turbulence_%s'%item].index

for i in turbulence_index:
    for j in range(120):
        turbulence_data[j].append(turbulence_all[i-j])
        
df=pd.DataFrame()
for j in range(120):
    df['speed_%s'%j]=speed_data[j]
    
df_power=pd.DataFrame()
for j in range(120):
    df_power['power_%s'%j]=power_data[j]

df_tem=pd.DataFrame()
for j in range(120):
    df_tem['tem_%s'%j]=tem_data[j]

df_angle=pd.DataFrame()
for j in range(120):
    df_angle['angle_%s'%j]=angle_data[j]

df_turbulence=pd.DataFrame()
for j in range(120):
    df_turbulence['turbulence_%s'%j]=turbulence_data[j]

speed_df=pd.DataFrame()
speed_df['speed0']=df['speed_0']
speed_df['power1']=df_power['power_1']
speed_df['tem_0']=df_tem['tem_0']
speed_df['angle_0']=df_angle['angle_0']
speed_df['turbulence_0']=df_turbulence['turbulence_0']

speed_df['speed1']=df['speed_1']
speed_df['power2']=df_power['power_2']
speed_df['tem_1']=df_tem['tem_1']
speed_df['angle_1']=df_angle['angle_1']
speed_df['turbulence_1']=df_turbulence['turbulence_1']

speed_df['speed2']=df['speed_2']
speed_df['power3']=df_power['power_3']
speed_df['tem_2']=df_tem['tem_2']
speed_df['angle_2']=df_angle['angle_2']
speed_df['turbulence_2']=df_turbulence['turbulence_2']

speed_df['speed3']=df['speed_3']
speed_df['power4']=df_power['power_4']
speed_df['tem_3']=df_tem['tem_3']
speed_df['angle_3']=df_angle['angle_3']
speed_df['turbulence_3']=df_turbulence['turbulence_3']

speed_df['turbulence_4']=df_turbulence['turbulence_4']

'\n#进行u-sd标准化\ncol_name=speed_df.columns\nfor item in col_name:\n    tmp=np.array(speed_df[item])\n    tmp_1=(tmp-np.mean(tmp))/np.std(tmp)\n    speed_df[item]=tmp_1\n\npower=(power-np.min(power))/(np.max(power)-np.min(power))\n'

In [ ]:
speed_df=pd.DataFrame()
power=(power-np.min(power))/(np.max(power)-np.min(power))

for i in range(4):
  speed_df['power%s'%(i+1)]=df_power['power_%s'%(i+1)]
  speed_df['speed%s'%i]=df['speed_%s'%i]
  speed_df['angle_%s'%i]=df_angle['angle_%s'%i]
  speed_df['tem_%s'%i]=df_tem['tem_%s'%i]

In [ ]:
#Skewness and kurtosis analysis
col_name=speed_df.columns
for item in col_name:
    tmp=np.array(speed_df[item])
    tmp_1=(tmp-np.mean(tmp))/np.std(tmp)
    speed_df[item]=tmp_1
    
speed=np.array(speed_df)

X_train, X_test, y_train, y_test = train_test_split(speed,power, test_size=0.2,)
X_train_1=X_train
X_test_1=X_test
X_train = np.reshape(X_train, (X_train.shape[0], 4, 4))
X_test = np.reshape(X_test, (X_test.shape[0], 4, 4))

In [ ]:
#LossHistory class
class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # # acc
        # plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'dodgerblue', label='train loss')
        if loss_type == 'epoch':
            # # val_acc
            # plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'darkorange', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('loss')
        plt.legend(loc="upper right")
        plt.savefig('loss{0}'.format(loss_type))

    def acc_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'dodgerblue', label='train acc')
        # loss
        # plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'darkorange', label='val acc')
            # val_loss
            # plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc')
        plt.legend(loc="upper right")
        plt.savefig('acc{0}'.format(loss_type))

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model
from keras.utils.vis_utils import plot_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform,RandomNormal
from matplotlib import pyplot as plt

In [ ]:
TIME_STEPS=4
INPUT_DIM=4
SINGLE_ATTENTION_VECTOR=False

def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    print(input_dim)
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='attention')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

def get_layer_output(model,layer_name,inputs):
    layer = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    layer_out = layer.predict(inputs)
    return np.mean(layer_out,axis=0)

def create_lstm_attention_model():
  mean_inputs = tf.keras.Input((TIME_STEPS,INPUT_DIM))
  attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(64, return_sequences=True)(attention_mul)
  mean_x=LSTM(64, return_sequences=False)(mean_x)
  mean_x = Dense(32,)(mean_x)
  mean_x=Dense(1)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=mean_x)
  return model

def create_lstm_attention_model():
  mean_inputs = tf.keras.Input((TIME_STEPS,INPUT_DIM))
  attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(128, return_sequences=True)(attention_mul)
  mean_x=LSTM(64, return_sequences=False)(mean_x)
  mean_x = Dense(64,)(mean_x)
  mean_x=Dense(1)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=mean_x)
  return model

In [ ]:
num_epochs = 50
mean_model = create_lstm_attention_model()

speed=np.array(speed_df)
X_train, X_test, y_train, y_test = train_test_split(speed,power, test_size=0.2,)
X_train_1=X_train
X_test_1=X_test
X_train = np.reshape(X_train, (X_train.shape[0], 4, 4))
X_test = np.reshape(X_test, (X_test.shape[0], 4, 4))

history = LossHistory()
mean_model.compile(optimizer='adam',loss='mean_squared_error',metrics=[keras.metrics.RootMeanSquaredError()])#loss,learning_rate=learning_rate;keras.optimizers.RMSprop()

mean_model.fit(X_train,y_train,batch_size=64, epochs=num_epochs,validation_split=0.15,callbacks=[history])    

### Whether using historical data to analyze

In [ ]:
np.shape(y_train)

(24148,)

In [ ]:
%%time

TIME_STEPS=4
INPUT_DIM=4
SINGLE_ATTENTION_VECTOR=False

def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    print(input_dim)
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='attention')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

def create_lstm_attention_model_notime():
  mean_inputs = tf.keras.Input((1,3))
  mean_x = LSTM(64, return_sequences=True)(mean_inputs)
  mean_x=LSTM(64, return_sequences=False)(mean_x)
  mean_x = Dense(32,)(mean_x)
  mean_x=Dense(1)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=mean_x)
  return model

def create_lstm_attention_model_withtime():
  mean_inputs = tf.keras.Input((TIME_STEPS,INPUT_DIM))
  mean_x = LSTM(64, return_sequences=True)(mean_inputs)
  mean_x=LSTM(64, return_sequences=False)(mean_x)
  mean_x = Dense(32,)(mean_x)
  mean_x=Dense(1)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=mean_x)
  return model

def create_lstm_attention_model_attention():
  mean_inputs = tf.keras.Input((TIME_STEPS,INPUT_DIM))
  attention_mul = attention_3d_block(mean_inputs)
  mean_x = LSTM(64, return_sequences=True)(attention_mul)
  mean_x=LSTM(64, return_sequences=False)(mean_x)
  mean_x = Dense(32,)(mean_x)
  mean_x=Dense(1)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=mean_x)
  return model
  
X_train, X_test, y_train, y_test = train_test_split(speed_df,power, test_size=0.2,)
X_train_2, X_test_2 = X_train[['speed0','angle_0','tem_0']],X_test[['speed0','angle_0','tem_0']]


model1=create_lstm_attention_model_withtime()
model2=create_lstm_attention_model_notime()
model3=create_lstm_attention_model_attention()

X_train=np.array(X_train)
X_test=np.array(X_test)

X_train = np.reshape(X_train, (X_train.shape[0], 4, 4))
X_test = np.reshape(X_test, (X_test.shape[0], 4, 4))

X_train_2=np.array(X_train_2)
X_test_2=np.array(X_test_2)

X_train_2 = np.reshape(X_train_2, (X_train_2.shape[0], 1, 3))
X_test_2 = np.reshape(X_test_2, (X_test_2.shape[0], 1, 3))


history3 = LossHistory()
model3.compile(optimizer='adam',loss='mean_squared_error',metrics=[keras.metrics.RootMeanSquaredError()])#loss,learning_rate=learning_rate;keras.optimizers.RMSprop()
model3.fit(X_train,y_train,batch_size=64, epochs=50,validation_split=0.15,callbacks=[history3])

history1 = LossHistory()
model1.compile(optimizer='adam',loss='mean_squared_error',metrics=[keras.metrics.RootMeanSquaredError()])#loss,learning_rate=learning_rate;keras.optimizers.RMSprop()
model1.fit(X_train,y_train,batch_size=64, epochs=50,validation_split=0.15,callbacks=[history1])

history2 = LossHistory()
model2.compile(optimizer='adam',loss='mean_squared_error',metrics=[keras.metrics.RootMeanSquaredError()])#loss,learning_rate=learning_rate;keras.optimizers.RMSprop()
model2.fit(X_train_2,y_train,batch_size=64, epochs=50,validation_split=0.15,callbacks=[history1])

In [ ]:
from sklearn.metrics import mean_squared_error as MSE,mean_absolute_error as MAE,max_error as ME,r2_score 
from sklearn import metrics

#preds = mean_model.predict(X_test)
prediction_mean = model2(X_test_2)

r_2=r2_score(prediction_mean,y_test)
rmse = np.sqrt(MSE(prediction_mean, y_test))
mae = MAE(prediction_mean, y_test)
mape_ = metrics.mean_absolute_percentage_error(prediction_mean, y_test)
mse=MSE(prediction_mean, y_test)

print('rmse:%s'%rmse)
print('mae:%s'%mae)
print('r2:%s'%r_2)
print('mape:%s'%mape_)

rmse:0.04351730761019866
mae:0.02919223896158769
r2:0.9748938355682032
mape:0.14064352920746015


In [ ]:
att = get_layer_output(mean_model,'attention_vec',X_test)
print(att.flatten())
x = np.array(speed_df.columns)

sns.barplot(att.flatten(),x)
print(np.sum(att))

### Compared Experiment

XGBOOST

In [ ]:
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error as MSE,mean_absolute_error as MAE,max_error as ME,r2_score 
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
xgboost = XGBRegressor(gamma=0,                 
                 learning_rate=0.01,
                 max_depth=5,
                 min_child_weight=3,
                 n_estimators=300,                                                                    
                 reg_alpha=0,
                 reg_lambda=0,
                 subsample=0.8,
                 seed=42) 

In [ ]:
xgboost.fit(X_train_1,y_train)
preds= xgboost.predict(X_test_1)

[03:25:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
#preds = mean_model.predict(X_test)
prediction_mean = mean_model(X_test)

r_2=r2_score(preds,y_test)
rmse = np.sqrt(MSE(preds, y_test))
mae = MAE(preds, y_test)
mape_ = metrics.mean_absolute_percentage_error(preds, y_test)

print('rmse:%s'%rmse)
print('mae:%s'%mae)
print('r2:%s'%r_2)
print('mape:%s'%mape_)

rmse:0.051882507513318896
mae:0.035507519201023525
r2:0.9646321406362895
mape:0.2028401696389018


In [ ]:
Rforest = RandomForestRegressor(n_estimators=50, criterion='mse', max_depth=100, min_samples_split=2,
                                max_features='auto',)

Rforest.fit(X_train_1, y_train)
preds= Rforest.predict(X_test_1) 

cubic polynomial

In [ ]:
def polyfit(degree, x_train, y, x_test):
    poly_features = PolynomialFeatures(degree=degree)
    
    X_train_poly = poly_features.fit_transform(x_train)
    poly_model = LinearRegression()
    lm_poly = poly_model.fit(X_train_poly, y)
    
    R_squared = lm_poly.score(X_train_poly, y)
    predictions = lm_poly.predict(poly_features.fit_transform(x_test))
    
    return R_squared, predictions

r2, preds = polyfit(3, X_train_1, y_train, X_test_1)

KNN

In [ ]:
model = KNeighborsRegressor(n_neighbors = 25,weights='uniform',metric = 'minkowski',metric_params={'p': 3})
model.fit(X_train_1, y_train)  #fit the model
preds=model.predict(X_test_1) #make prediction on test set

DNN

In [ ]:
from keras.models import Sequential, load_model
from keras.initializers import he_normal
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def define_and_fit_model(layer_nodes, num_epochs, batch, optimiser, dropout_rate=0.1, input_nodes=16):

    num_layers = len(layer_nodes)
    
    ### Define Model ###
    model = Sequential()
    first = True
    for l in range(num_layers):
        if first:
            # Input layer
            model.add(Dense(layer_nodes[l], input_dim=input_nodes,
                            kernel_initializer=he_normal(seed=1),
                            # kernel_regularizer=regularizers.l2(0.1),
                            activation='relu'))
            first = False
            
        else:
            # Hidden layers
            model.add(Dense(layer_nodes[l],
                            kernel_initializer=he_normal(seed=1),
                            # kernel_regularizer=regularizers.l2(0.1),
                            activation='relu'))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))
            
    # Output layer
    model.add(Dense(1, kernel_initializer=he_normal(seed=1)))
    
    print(model.summary())
    
    ### Fit Model ###
    model.compile(loss='mean_squared_error', optimizer=optimiser)
    checkpoint = ModelCheckpoint("model.hdf5", monitor='val_loss', verbose=0,
                                 save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    history = model.fit(X_train_1, y_train, 
                        epochs=num_epochs, batch_size=batch, verbose=0,
                        validation_split=0.15,
                        callbacks=callbacks_list)

    run_codename = "run__{layers}__b{batch_size}__{opt}__d{drop}__e{epochs}".format(
        layers="_".join(map(str, layer_nodes)), batch_size='%04d' % batch, opt=optimiser, 
        drop=('%.4f' % 0.01).replace('.','_'), epochs= '%05d' % batch)
    
    return model, history, run_codename

In [ ]:
model, hist, name = define_and_fit_model(layer_nodes=[64], num_epochs=50, batch=64, optimiser='nadam')
preds = model.predict(X_test_1)

Heteroscedasticity spline

In [ ]:
import numpy as np
import pymc3 as pm
import theano.tensor as tt
import matplotlib.pyplot as plt
import arviz as az
from scipy.interpolate import BSpline

In [ ]:
speed_0=np.array(speed_df['speed0'])
speed_1=np.array(speed_df['speed1'])
power_1=np.array(speed_df['power1'])
power_2=np.array(speed_df['power2'])
power_0=np.array(norm_data['power(kW)_1'])#原始的power数据

In [ ]:
#多变量
NK = 12
knots_1 = np.linspace(speed_0[:10000].min(),speed_0[:10000].max(),NK)
knots_2 = np.linspace(power_1[:10000].min(),power_1[:10000].max(),NK)
# Univariate spline in the B-spline basis
basis_funcs_1 = BSpline(knots_1, np.eye(NK), k=3)  # cubic spline representation
basis_funcs_2 = BSpline(knots_2, np.eye(NK), k=3)  # cubic spline representation
Bx_1_1 = basis_funcs_1(speed_0[:10000])  # coefficients
Bx_1_2 = basis_funcs_1(speed_1[:10000])  # coefficients
Bx_2_1 = basis_funcs_2(power_1[:10000])  # coefficients
Bx_2_2 = basis_funcs_2(power_2[:10000])  # coefficients
Bx=np.c_[Bx_1_1,Bx_1_2,Bx_2_1,Bx_2_2]

In [ ]:
with pm.Model() as splines:
    # hyper-priors
    mu0 = pm.Normal('mu0', mu=0., sd=10.)
    sd0 = pm.HalfCauchy('sd0', beta=5.)
    # priors (must use the same NK for y1 & y2!)
    betai = pm.Normal('betai', mu=mu0, sd=sd0, shape=NK*4)
    sd = pm.HalfCauchy('sd', beta=10.)
    # link (spline regression)
    mu = pm.Deterministic('mu', pm.math.dot(Bx, betai))
    # likelihood
    yi = pm.Normal('yhat', mu=mu, sd=sd, observed=power[:10000])
    # MCMC
    start=pm.find_MAP()
    chain = pm.sample(draws=5000, tune=2000, chains=2,init = 'advi+adapt_diag',start=start)#,target_accept=0.95

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...


Convergence achieved at 80100
Interrupted at 80,099 [40%]: Average Loss = 7,769.9
Sequential sampling (2 chains in 1 job)
NUTS: [sd, betai, sd0, mu0]


Sampling 2 chains for 2_000 tune and 5_000 draw iterations (4_000 + 10_000 draws total) took 10944 seconds.
The acceptance probability does not match the target. It is 0.8808883038558398, but should be close to 0.8. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.


In [ ]:
betai_est = chain.get_values(varname='betai', burn=1000, thin=2, combine=True)
y_est = np.dot(Bx, betai_est.mean(axis=0))
plt.scatter(speed_0[:10000], power[:10000], c='red')
plt.scatter(speed_0[:10000], y_est, c='royalblue')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error as MSE,mean_absolute_error as MAE,max_error as ME,r2_score 
from sklearn import metrics

r_2=r2_score(y_est,power[10000:])
rmse = np.sqrt(MSE(y_est,power[10000:]))
mae = MAE(y_est,power[10000:])
mape_ = metrics.mean_absolute_percentage_error(y_est,power[10000:])
mse=MSE(y_est,power[10000:])

print('rmse:%s'%rmse)
print('mae:%s'%mae)
print('r2:%s'%r_2)
print('mape:%s'%mape_)

rmse:0.051497417911227424
mae:0.03542262070077466
r2:0.9672879995316568
mape:0.1346861475709001


In [ ]:
Bx_1_1 = basis_funcs_1(speed_0[10000:])  # coefficients
Bx_1_2 = basis_funcs_1(speed_1[10000:])  # coefficients
Bx_2_1 = basis_funcs_2(power_1[10000:])  # coefficients
Bx_2_2 = basis_funcs_2(power_2[10000:])  # coefficients
Bx_test=np.c_[Bx_1_1,Bx_1_2,Bx_2_1,Bx_2_2]

In [ ]:
y_est = np.dot(Bx_test, betai_est.mean(axis=0))